In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"   #(xxxx is your specific GPU ID)

In [2]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.layers import Activation, Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
from PIL import Image
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from MyEarlyStopping import MyEarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# from keras.optimizers import adam

import timeit
from sklearn.metrics import confusion_matrix
import sklearn.metrics as metrics
from sklearn.preprocessing import LabelEncoder  

# load dataset 

## Training dataset (40%)

In [3]:
train_data = ImageDataGenerator(preprocessing_function=preprocess_input)
train_generator = train_data.flow_from_directory('/data/home/cou/yuli5/semg/Audio_MEL_Data/train',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                               shuffle=False)

Found 1714 images belonging to 10 classes.


In [4]:
image_names = train_generator.filenames
image_no = [i.split("/")[1].split("_")[2].split(".")[0] for i in image_names]
image_no = np.array(list(map(int, image_no)))
ALL_participant_class = [i.split("/")[1].split("_")[1] for i in image_names]
ALL_participant_class = np.array(list(map(int, ALL_participant_class)))
command_class = train_generator.classes
All_command_class = tf.keras.utils.to_categorical(command_class, num_classes=10)

In [5]:
All_Inputs = [next(train_generator)[0][0] for _ in range(len(train_generator))]
All_Inputs = np.array(All_Inputs)

In [6]:
subject = 5
train_image = 10 #10:20%, 20: 40%, 30:60%

In [7]:
select_indexs_train = (image_no<train_image)&(ALL_participant_class==subject)
Train_Inputs = All_Inputs[select_indexs_train]
Train_command_class = All_command_class[select_indexs_train]
sum(Train_command_class)

array([10., 10., 10., 10., 10., 10., 10., 10., 10., 10.], dtype=float32)

## Validation dataset (20%)

In [8]:
val_data = ImageDataGenerator(preprocessing_function=preprocess_input)
val_generator = val_data.flow_from_directory('/data/home/cou/yuli5/semg/Audio_MEL_Data/validation',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                               shuffle=False)

Found 543 images belonging to 10 classes.


In [9]:
image_names = val_generator.filenames
participant_class = [i.split("/", 1)[1].split("_")[1] for i in image_names]
participant_class = np.array(list(map(int, participant_class)))
command_class = val_generator.classes
Val_command_class = tf.keras.utils.to_categorical(command_class, num_classes=10)


In [10]:
ALL_Val_Inputs = [next(val_generator)[0][0] for _ in range(len(val_generator))]
ALL_Val_Inputs = np.array(ALL_Val_Inputs)

In [11]:
select_indexs_val = participant_class==subject
Val_Inputs = ALL_Val_Inputs[select_indexs_val]
Val_command_class = Val_command_class[select_indexs_val]
sum(Val_command_class)

array([10., 11., 10.,  9., 11., 10., 11.,  8.,  9., 11.], dtype=float32)

## Test dataset (20%)

In [12]:
test_data = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_data.flow_from_directory('/data/home/cou/yuli5/semg/Audio_MEL_Data/test',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                                shuffle=False)

Found 543 images belonging to 10 classes.


In [13]:
image_names = test_generator.filenames
participant_class = [i.split("/", 1)[1].split("_")[1] for i in image_names]
test_unit_participant_class = np.array(list(map(int, participant_class)))
test_unit_command_class = test_generator.classes
Test_command_class = tf.keras.utils.to_categorical(test_unit_command_class, num_classes=10)

In [14]:
Test_Inputs = [next(test_generator)[0][0] for _ in range(len(test_generator))]
Test_Inputs = np.array(Test_Inputs)

In [15]:
Test_Inputs_1 = Test_Inputs[np.where(test_unit_participant_class == 1)]
Test_command_class_1 = Test_command_class[np.where(test_unit_participant_class == 1)]
Test_Inputs_2 = Test_Inputs[np.where(test_unit_participant_class == 2)]
Test_command_class_2 = Test_command_class[np.where(test_unit_participant_class == 2)]
Test_Inputs_3 = Test_Inputs[np.where(test_unit_participant_class == 3)]
Test_command_class_3 = Test_command_class[np.where(test_unit_participant_class == 3)]
Test_Inputs_4 = Test_Inputs[np.where(test_unit_participant_class == 4)]
Test_command_class_4 = Test_command_class[np.where(test_unit_participant_class == 4)]
Test_Inputs_5 = Test_Inputs[np.where(test_unit_participant_class == 5)]
Test_command_class_5 = Test_command_class[np.where(test_unit_participant_class == 5)]

In [16]:
# sum(Test_command_class_2)

## Speaker 6 Test dataset 

In [17]:
test_data_6 = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator_6 = test_data_6.flow_from_directory('/data/home/cou/yuli5/semg/Audio_MEL_subject/p_6_split/test',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                                shuffle=False)

Found 100 images belonging to 10 classes.


In [18]:
image_names = test_generator_6.filenames
command_class = [i.split("/", 1)[0]for i in image_names]
le = LabelEncoder()
test_unit_command_class_6 = le.fit_transform(command_class)
Test_command_class_6 = tf.keras.utils.to_categorical(test_unit_command_class_6, num_classes=10)

In [19]:
Test_Inputs_6 = [next(test_generator_6)[0][0] for _ in range(len(test_generator_6))]
Test_Inputs_6 = np.array(Test_Inputs_6)

## Speaker 7 Test dataset 

In [20]:
test_data_7 = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator_7 = test_data_6.flow_from_directory('/data/home/cou/yuli5/semg/Audio_MEL_subject/p_7_split/test',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                                shuffle=False)

Found 125 images belonging to 10 classes.


In [21]:
image_names = test_generator_7.filenames
command_class = [i.split("/", 1)[0]for i in image_names]
le = LabelEncoder()
test_unit_command_class_7 = le.fit_transform(command_class)
Test_command_class_7 = tf.keras.utils.to_categorical(test_unit_command_class_7, num_classes=10)

In [22]:
Test_Inputs_7 = [next(test_generator_7)[0][0] for _ in range(len(test_generator_7))]
Test_Inputs_7 = np.array(Test_Inputs_7)

## Speaker 8 Test dataset 

In [23]:
test_data_8 = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator_8 = test_data_6.flow_from_directory('/data/home/cou/yuli5/semg/Audio_MEL_subject/p_8_split/test',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                                shuffle=False)

Found 101 images belonging to 10 classes.


In [24]:
image_names = test_generator_8.filenames
command_class = [i.split("/", 1)[0]for i in image_names]
le = LabelEncoder()
test_unit_command_class_8 = le.fit_transform(command_class)
Test_command_class_8 = tf.keras.utils.to_categorical(test_unit_command_class_8, num_classes=10)

In [25]:
Test_Inputs_8 = [next(test_generator_8)[0][0] for _ in range(len(test_generator_8))]
Test_Inputs_8 = np.array(Test_Inputs_8)

# pd file store performance

In [26]:
# pd file store performance
# Perfomance = pd.DataFrame(columns = ['Model','Time','Command_Acc_p15','Command_Acc_p1','Command_Acc_p2',
#                                      'Command_Acc_p3','Command_Acc_p4','Command_Acc_p5','Acc_p6','Acc_p7','Acc_p8'])

Perfomance = pd.read_csv('Performance_0608_single_model.csv')
# Perfomance
Perfomance                                                        

,Model,Time,Command_Acc_p15,Command_Acc_p1,Command_Acc_p2,Command_Acc_p3,Command_Acc_p4,Command_Acc_p5,Acc_p6,Acc_p7,Acc_p8,Size
0,Subjet_1,15.269878,0.6243,0.9369,0.3889,0.5391,0.6239,0.63,0.38,0.384,0.2673,10
1,Subjet_1,12.062418,0.5709,0.9189,0.3148,0.4696,0.5413,0.61,0.33,0.320,0.2673,10
2,Subjet_1,13.520935,0.6335,0.9550,0.3889,0.5478,0.6514,0.62,0.48,0.384,0.2475,10
3,Subjet_1,14.664869,0.6225,0.9550,0.2685,0.5217,0.6789,0.69,0.26,0.456,0.2970,10
4,Subjet_1,13.347304,0.6188,0.9279,0.3796,0.5652,0.5780,0.64,0.39,0.416,0.2574,10
...,...,...,...,...,...,...,...,...,...,...,...,...
115,Subjet_4,15.352048,0.6832,0.6126,0.5000,0.6435,0.9908,0.67,0.31,0.240,0.2970,30
116,Subjet_4,16.284204,0.6667,0.5676,0.3889,0.6348,0.9908,0.76,0.30,0.232,0.2673,30
117,Subjet_4,15.602407,0.6667,0.5676,0.4352,0.6087,0.9908,0.74,0.35,0.232,0.2772,30
118,Subjet_4,17.284093,0.7127,0.6396,0.5370,0.6522,0.9908,0.75,0.29,0.216,0.2772,30


# Create model 

In [27]:
actv_fun_1_1 = "relu" 
actv_fun_1_2 = "sigmoid"
shape_1_1 = 128
shape_1_2 = 256
actv_fun_2_1 = "sigmoid" 
actv_fun_2_2 = "sigmoid"
shape_2_1 = 512
shape_2_2 = 512


## Single model （size:10）

In [28]:
high_acc = 0
for run in range(0,10):
    # feature extraction layers
    resnet_model = ResNet50(input_shape=(224, 224,3), include_top=False, weights="imagenet")
    for layer in resnet_model.layers:
        layer.trainable = False

    d = resnet_model.output.shape[-1] # dimension of last layer


    ###################### model 2 ###################### 
    layer_2_0 = tf.keras.layers.Dense(d,name="weight_2")(resnet_model.output) #times weight before flatten
    layer_2_1 = tf.keras.layers.Flatten(name='flatten_2')(layer_2_0)

    Dense_2_1 = tf.keras.layers.Dense(shape_2_1, activation=actv_fun_2_1,name='fc2_1')
    layer_2_2  = Dense_2_1(layer_2_1)
    Dense_2_2 = tf.keras.layers.Dense(shape_2_2, activation=actv_fun_2_2,name='fc2_2')
    layer_2_3  = Dense_2_2(layer_2_2)

    Dense_2_3 = tf.keras.layers.Dense(10, activation='softmax',name='command_output')
    out_layer_2 = Dense_2_3(layer_2_3)


    resnet_model = tf.keras.Model(resnet_model.input, out_layer_2)


    # resnet_model.summary() 


    ##################### training ############################
    opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
    callbacks = [EarlyStopping(monitor = 'val_accuracy', patience=5,restore_best_weights=True)]
    resnet_model.compile(optimizer=opt, loss="categorical_crossentropy",metrics=['accuracy'])

    
    start = timeit.default_timer()
       
    history1 = resnet_model.fit(Train_Inputs,Train_command_class,
                                validation_data=(Val_Inputs,Val_command_class),
                                callbacks=callbacks,
                                batch_size=64,
                                epochs=10)
    
    for layer in resnet_model.layers[0:175]:
        layer.trainable = True
        
    history2 = resnet_model.fit(Train_Inputs,Train_command_class,
                            validation_data=(Val_Inputs,Val_command_class), 
                           callbacks=callbacks,
                           batch_size=64,
                           epochs=10)

    stop = timeit.default_timer()
    run_time = stop - start
    
    ##################### test performance ############################
    predictions = resnet_model.predict(Test_Inputs)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class
    acc_p15_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)


    # test on p1
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_1)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_1, axis=-1)
    acc_p1_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p2
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_2)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_2, axis=-1)
    acc_p2_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p3 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_3)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_3, axis=-1)
    acc_p3_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p4
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_4)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_4, axis=-1)
    acc_p4_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p5

    ## command classification
    predictions = resnet_model.predict(Test_Inputs_5)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_5, axis=-1)
    acc_p5_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p6 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_6)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class_6
    acc_p6 = metrics.accuracy_score(true_classes, predicted_classes).round(4)                

    # test on p7 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_7)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class_7
    acc_p7 = metrics.accuracy_score(true_classes, predicted_classes).round(4)    

    # test on p8 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_8)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class_8
    acc_p8 = metrics.accuracy_score(true_classes, predicted_classes).round(4)    

    Perfomance = Perfomance.append({'Model': "Subjet_5",'Size':'10','Time':run_time,'Command_Acc_p15':acc_p15_c,
                                    'Command_Acc_p1':acc_p1_c,'Command_Acc_p2':acc_p2_c,
                                    'Command_Acc_p3':acc_p3_c,'Command_Acc_p4':acc_p4_c,'Command_Acc_p5':acc_p5_c,
                                    'Acc_p6':acc_p6,'Acc_p7':acc_p7,'Acc_p8':acc_p8}, ignore_index=True)


    if high_acc < acc_p15_c :
        resnet_model.save('Initial_subject_5_model_size10_0608.h5')
        high_acc = acc_p15_c 
        best_index = run
        pd.DataFrame.from_dict(history1.history).to_csv('subject_5_size10_history1_0608.csv',index=False)
        pd.DataFrame.from_dict(history2.history).to_csv('subject_5_size10_history2_0608.csv',index=False)
        
    del resnet_model
    run = run + 1

Epoch 1/10
2/2 [==============================] - 2s 822ms/step - loss: 2.4632 - accuracy: 0.1000 - val_loss: 2.2202 - val_accuracy: 0.1200
Epoch 2/10
2/2 [==============================] - 1s 306ms/step - loss: 2.1453 - accuracy: 0.2400 - val_loss: 1.9787 - val_accuracy: 0.5000
Epoch 3/10
2/2 [==============================] - 1s 320ms/step - loss: 1.9130 - accuracy: 0.5800 - val_loss: 1.8442 - val_accuracy: 0.6500
Epoch 4/10
2/2 [==============================] - 1s 285ms/step - loss: 1.7665 - accuracy: 0.7400 - val_loss: 1.7569 - val_accuracy: 0.7300
Epoch 5/10
2/2 [==============================] - 1s 286ms/step - loss: 1.6731 - accuracy: 0.8100 - val_loss: 1.6696 - val_accuracy: 0.8400
Epoch 6/10
2/2 [==============================] - 1s 286ms/step - loss: 1.5858 - accuracy: 0.9700 - val_loss: 1.6129 - val_accuracy: 0.9100
Epoch 7/10
2/2 [==============================] - 1s 270ms/step - loss: 1.5220 - accuracy: 0.9800 - val_loss: 1.5631 - val_accuracy: 0.9200
Epoch 8/10
2/2 [====

2/2 [==============================] - 1s 291ms/step - loss: 1.8527 - accuracy: 0.8300 - val_loss: 1.8063 - val_accuracy: 0.8400
Epoch 4/10
2/2 [==============================] - 0s 90ms/step - loss: 1.7209 - accuracy: 0.8900 - val_loss: 1.7114 - val_accuracy: 0.8300
Epoch 5/10
2/2 [==============================] - 1s 290ms/step - loss: 1.6205 - accuracy: 0.9100 - val_loss: 1.6306 - val_accuracy: 0.9200
Epoch 6/10
2/2 [==============================] - 1s 280ms/step - loss: 1.5426 - accuracy: 0.9800 - val_loss: 1.5682 - val_accuracy: 0.9400
Epoch 7/10
2/2 [==============================] - 1s 278ms/step - loss: 1.4786 - accuracy: 0.9900 - val_loss: 1.5208 - val_accuracy: 0.9500
Epoch 8/10
2/2 [==============================] - 0s 87ms/step - loss: 1.4306 - accuracy: 0.9900 - val_loss: 1.4812 - val_accuracy: 0.9400
Epoch 9/10
2/2 [==============================] - 0s 85ms/step - loss: 1.3884 - accuracy: 0.9900 - val_loss: 1.4397 - val_accuracy: 0.9400
Epoch 10/10
2/2 [=================

Epoch 6/10
2/2 [==============================] - 0s 90ms/step - loss: 1.5969 - accuracy: 0.9800 - val_loss: 1.6170 - val_accuracy: 0.9500
Epoch 7/10
2/2 [==============================] - 0s 90ms/step - loss: 1.5351 - accuracy: 1.0000 - val_loss: 1.5692 - val_accuracy: 0.9600
Epoch 8/10
2/2 [==============================] - 0s 99ms/step - loss: 1.4863 - accuracy: 1.0000 - val_loss: 1.5332 - val_accuracy: 0.9600
Epoch 9/10
2/2 [==============================] - 1s 273ms/step - loss: 1.4477 - accuracy: 1.0000 - val_loss: 1.4947 - val_accuracy: 0.9600
Epoch 1/10
2/2 [==============================] - 1s 338ms/step - loss: 1.6547 - accuracy: 0.9800 - val_loss: 1.6447 - val_accuracy: 0.9500
Epoch 2/10
2/2 [==============================] - 0s 90ms/step - loss: 1.5622 - accuracy: 1.0000 - val_loss: 1.5955 - val_accuracy: 0.9500
Epoch 3/10
2/2 [==============================] - 0s 94ms/step - loss: 1.5080 - accuracy: 1.0000 - val_loss: 1.5460 - val_accuracy: 0.9500
Epoch 4/10
2/2 [=========

2/2 [==============================] - 0s 91ms/step - loss: 1.5896 - accuracy: 0.9000 - val_loss: 1.6229 - val_accuracy: 0.8300
Epoch 9/10
2/2 [==============================] - 1s 290ms/step - loss: 1.5494 - accuracy: 0.9100 - val_loss: 1.5859 - val_accuracy: 0.8800
Epoch 10/10
2/2 [==============================] - 1s 296ms/step - loss: 1.5115 - accuracy: 1.0000 - val_loss: 1.5558 - val_accuracy: 0.9400
Epoch 1/10
2/2 [==============================] - 1s 307ms/step - loss: 1.4825 - accuracy: 1.0000 - val_loss: 1.5250 - val_accuracy: 0.9500
Epoch 2/10
2/2 [==============================] - 1s 284ms/step - loss: 1.4551 - accuracy: 1.0000 - val_loss: 1.4938 - val_accuracy: 0.9600
Epoch 3/10
2/2 [==============================] - 0s 94ms/step - loss: 1.4239 - accuracy: 1.0000 - val_loss: 1.4623 - val_accuracy: 0.9500
Epoch 4/10
2/2 [==============================] - 0s 94ms/step - loss: 1.3945 - accuracy: 1.0000 - val_loss: 1.4360 - val_accuracy: 0.9500
Epoch 5/10
2/2 [=================

In [29]:
resnet_model = tf.keras.models.load_model('Initial_subject_5_model_size10_0608.h5')
predictions = resnet_model.predict(Test_Inputs_5)
predicted_classes =  np.argmax(predictions, axis=1)
acc_c = round(sum(x == y for x, y in zip(np.argmax(Test_command_class_5, axis=1), predicted_classes)) / len(np.argmax(Test_command_class_5, axis=1)),4)
print(acc_c)

0.95


In [30]:
Perfomance

,Model,Time,Command_Acc_p15,Command_Acc_p1,Command_Acc_p2,Command_Acc_p3,Command_Acc_p4,Command_Acc_p5,Acc_p6,Acc_p7,Acc_p8,Size
0,Subjet_1,15.269878,0.6243,0.9369,0.3889,0.5391,0.6239,0.63,0.38,0.384,0.2673,10
1,Subjet_1,12.062418,0.5709,0.9189,0.3148,0.4696,0.5413,0.61,0.33,0.320,0.2673,10
2,Subjet_1,13.520935,0.6335,0.9550,0.3889,0.5478,0.6514,0.62,0.48,0.384,0.2475,10
3,Subjet_1,14.664869,0.6225,0.9550,0.2685,0.5217,0.6789,0.69,0.26,0.456,0.2970,10
4,Subjet_1,13.347304,0.6188,0.9279,0.3796,0.5652,0.5780,0.64,0.39,0.416,0.2574,10
...,...,...,...,...,...,...,...,...,...,...,...,...
125,Subjet_5,12.167129,0.5930,0.4775,0.3241,0.5565,0.6697,0.97,0.24,0.312,0.3168,10
126,Subjet_5,12.007169,0.5985,0.5135,0.2963,0.5652,0.6881,0.96,0.21,0.336,0.3465,10
127,Subjet_5,12.707934,0.5930,0.4685,0.3241,0.5739,0.6606,0.97,0.24,0.256,0.2871,10
128,Subjet_5,13.188740,0.5875,0.4414,0.3148,0.5478,0.7248,0.94,0.25,0.248,0.3267,10


In [31]:
best_index

4

## Single model （size:20）

In [32]:
subject = 5
train_image = 20 #10:20%, 20: 40%, 30:60%

In [33]:
select_indexs_train = (image_no<train_image)&(ALL_participant_class==subject)
Train_Inputs = All_Inputs[select_indexs_train]
Train_command_class = All_command_class[select_indexs_train]
sum(Train_command_class)

array([20., 20., 20., 20., 20., 20., 20., 20., 20., 20.], dtype=float32)

In [34]:
high_acc = 0
for run in range(0,10):
    # feature extraction layers
    resnet_model = ResNet50(input_shape=(224, 224,3), include_top=False, weights="imagenet")
    for layer in resnet_model.layers:
        layer.trainable = False

    d = resnet_model.output.shape[-1] # dimension of last layer


    ###################### model 2 ###################### 
    layer_2_0 = tf.keras.layers.Dense(d,name="weight_2")(resnet_model.output) #times weight before flatten
    layer_2_1 = tf.keras.layers.Flatten(name='flatten_2')(layer_2_0)

    Dense_2_1 = tf.keras.layers.Dense(shape_2_1, activation=actv_fun_2_1,name='fc2_1')
    layer_2_2  = Dense_2_1(layer_2_1)
    Dense_2_2 = tf.keras.layers.Dense(shape_2_2, activation=actv_fun_2_2,name='fc2_2')
    layer_2_3  = Dense_2_2(layer_2_2)

    Dense_2_3 = tf.keras.layers.Dense(10, activation='softmax',name='command_output')
    out_layer_2 = Dense_2_3(layer_2_3)


    resnet_model = tf.keras.Model(resnet_model.input, out_layer_2)


    # resnet_model.summary() 


    ##################### training ############################
    opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
    callbacks = [EarlyStopping(monitor = 'val_accuracy', patience=5,restore_best_weights=True)]
    resnet_model.compile(optimizer=opt, loss="categorical_crossentropy",metrics=['accuracy'])

    
    start = timeit.default_timer()
       
    history1 = resnet_model.fit(Train_Inputs,Train_command_class,
                                validation_data=(Val_Inputs,Val_command_class),
                                callbacks=callbacks,
                                batch_size=64,
                                epochs=10)
    
    for layer in resnet_model.layers[0:175]:
        layer.trainable = True
        
    history2 = resnet_model.fit(Train_Inputs,Train_command_class,
                            validation_data=(Val_Inputs,Val_command_class), 
                           callbacks=callbacks,
                           batch_size=64,
                           epochs=10)

    stop = timeit.default_timer()
    run_time = stop - start
    
    ##################### test performance ############################
    predictions = resnet_model.predict(Test_Inputs)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class
    acc_p15_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)


    # test on p1
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_1)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_1, axis=-1)
    acc_p1_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p2
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_2)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_2, axis=-1)
    acc_p2_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p3 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_3)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_3, axis=-1)
    acc_p3_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p4
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_4)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_4, axis=-1)
    acc_p4_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p5

    ## command classification
    predictions = resnet_model.predict(Test_Inputs_5)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_5, axis=-1)
    acc_p5_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p6 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_6)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class_6
    acc_p6 = metrics.accuracy_score(true_classes, predicted_classes).round(4)                

    # test on p7 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_7)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class_7
    acc_p7 = metrics.accuracy_score(true_classes, predicted_classes).round(4)    

    # test on p8 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_8)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class_8
    acc_p8 = metrics.accuracy_score(true_classes, predicted_classes).round(4)    

    Perfomance = Perfomance.append({'Model': "Subjet_5",'Size':'20','Time':run_time,'Command_Acc_p15':acc_p15_c,
                                    'Command_Acc_p1':acc_p1_c,'Command_Acc_p2':acc_p2_c,
                                    'Command_Acc_p3':acc_p3_c,'Command_Acc_p4':acc_p4_c,'Command_Acc_p5':acc_p5_c,
                                    'Acc_p6':acc_p6,'Acc_p7':acc_p7,'Acc_p8':acc_p8}, ignore_index=True)


    if high_acc < acc_p15_c :
        resnet_model.save('Initial_subject_5_model_size20_0608.h5')
        high_acc = acc_p15_c 
        best_index = run
        pd.DataFrame.from_dict(history1.history).to_csv('subject_5_size20_history1_0608.csv',index=False)
        pd.DataFrame.from_dict(history2.history).to_csv('subject_5_size20_history2_0608.csv',index=False)
        
    del resnet_model
    run = run + 1

Epoch 1/10
4/4 [==============================] - 2s 409ms/step - loss: 2.4781 - accuracy: 0.2050 - val_loss: 2.0981 - val_accuracy: 0.5700
Epoch 2/10
4/4 [==============================] - 1s 173ms/step - loss: 2.0408 - accuracy: 0.6300 - val_loss: 1.9510 - val_accuracy: 0.6800
Epoch 3/10
4/4 [==============================] - 1s 204ms/step - loss: 1.9226 - accuracy: 0.7250 - val_loss: 1.8773 - val_accuracy: 0.7000
Epoch 4/10
4/4 [==============================] - 1s 173ms/step - loss: 1.8343 - accuracy: 0.7700 - val_loss: 1.8195 - val_accuracy: 0.8200
Epoch 5/10
4/4 [==============================] - 1s 166ms/step - loss: 1.7831 - accuracy: 0.8850 - val_loss: 1.7709 - val_accuracy: 0.8600
Epoch 6/10
4/4 [==============================] - 0s 75ms/step - loss: 1.7396 - accuracy: 0.9200 - val_loss: 1.7431 - val_accuracy: 0.8300
Epoch 7/10
4/4 [==============================] - 1s 176ms/step - loss: 1.7037 - accuracy: 0.8900 - val_loss: 1.7131 - val_accuracy: 0.8700
Epoch 8/10
4/4 [=====

Epoch 7/10
4/4 [==============================] - 0s 77ms/step - loss: 1.6393 - accuracy: 0.9950 - val_loss: 1.6448 - val_accuracy: 0.9600
Epoch 8/10
4/4 [==============================] - 1s 173ms/step - loss: 1.6005 - accuracy: 0.9950 - val_loss: 1.6048 - val_accuracy: 0.9700
Epoch 9/10
4/4 [==============================] - 1s 173ms/step - loss: 1.5608 - accuracy: 0.9950 - val_loss: 1.5646 - val_accuracy: 0.9800
Epoch 10/10
4/4 [==============================] - 0s 77ms/step - loss: 1.5204 - accuracy: 0.9950 - val_loss: 1.5246 - val_accuracy: 0.9800
Epoch 1/10
4/4 [==============================] - 1s 190ms/step - loss: 1.4803 - accuracy: 0.9950 - val_loss: 1.4884 - val_accuracy: 0.9800
Epoch 2/10
4/4 [==============================] - 0s 78ms/step - loss: 1.4422 - accuracy: 0.9950 - val_loss: 1.4581 - val_accuracy: 0.9800
Epoch 3/10
4/4 [==============================] - 0s 77ms/step - loss: 1.4062 - accuracy: 0.9950 - val_loss: 1.4282 - val_accuracy: 0.9800
Epoch 4/10
4/4 [=======

Epoch 4/10
4/4 [==============================] - 0s 74ms/step - loss: 1.0496 - accuracy: 1.0000 - val_loss: 1.0731 - val_accuracy: 0.9900
Epoch 5/10
4/4 [==============================] - 0s 77ms/step - loss: 1.0033 - accuracy: 1.0000 - val_loss: 1.0278 - val_accuracy: 0.9900
Epoch 6/10
4/4 [==============================] - 0s 83ms/step - loss: 0.9619 - accuracy: 1.0000 - val_loss: 0.9866 - val_accuracy: 0.9900
Epoch 7/10
4/4 [==============================] - 1s 162ms/step - loss: 0.9218 - accuracy: 1.0000 - val_loss: 0.9412 - val_accuracy: 0.9900
Epoch 1/10
4/4 [==============================] - 2s 376ms/step - loss: 2.4662 - accuracy: 0.2750 - val_loss: 2.0836 - val_accuracy: 0.6400
Epoch 2/10
4/4 [==============================] - 1s 175ms/step - loss: 2.0300 - accuracy: 0.6600 - val_loss: 1.9516 - val_accuracy: 0.6900
Epoch 3/10
4/4 [==============================] - 1s 168ms/step - loss: 1.9114 - accuracy: 0.7550 - val_loss: 1.8753 - val_accuracy: 0.7500
Epoch 4/10
4/4 [=======

In [35]:
resnet_model = tf.keras.models.load_model('Initial_subject_5_model_size20_0608.h5')
predictions = resnet_model.predict(Test_Inputs_5)
predicted_classes =  np.argmax(predictions, axis=1)
acc_c = round(sum(x == y for x, y in zip(np.argmax(Test_command_class_5, axis=1), predicted_classes)) / len(np.argmax(Test_command_class_5, axis=1)),4)
print(acc_c)

1.0


In [36]:
Perfomance

,Model,Time,Command_Acc_p15,Command_Acc_p1,Command_Acc_p2,Command_Acc_p3,Command_Acc_p4,Command_Acc_p5,Acc_p6,Acc_p7,Acc_p8,Size
0,Subjet_1,15.269878,0.6243,0.9369,0.3889,0.5391,0.6239,0.63,0.38,0.384,0.2673,10
1,Subjet_1,12.062418,0.5709,0.9189,0.3148,0.4696,0.5413,0.61,0.33,0.320,0.2673,10
2,Subjet_1,13.520935,0.6335,0.9550,0.3889,0.5478,0.6514,0.62,0.48,0.384,0.2475,10
3,Subjet_1,14.664869,0.6225,0.9550,0.2685,0.5217,0.6789,0.69,0.26,0.456,0.2970,10
4,Subjet_1,13.347304,0.6188,0.9279,0.3796,0.5652,0.5780,0.64,0.39,0.416,0.2574,10
...,...,...,...,...,...,...,...,...,...,...,...,...
135,Subjet_5,14.967050,0.6483,0.5495,0.3426,0.6870,0.6881,1.00,0.31,0.344,0.2673,20
136,Subjet_5,13.265092,0.5617,0.4595,0.2407,0.5478,0.6055,0.99,0.33,0.336,0.2673,20
137,Subjet_5,14.452641,0.6096,0.5045,0.3333,0.5739,0.6972,0.97,0.20,0.272,0.3465,20
138,Subjet_5,16.581088,0.6446,0.5856,0.3056,0.6348,0.7339,0.99,0.34,0.360,0.2475,20


In [37]:
best_index

5

## Single model （size:30）

In [38]:
subject = 5
train_image = 30 #10:20%, 20: 40%, 30:60%

In [39]:
select_indexs_train = (image_no<train_image)&(ALL_participant_class==subject)
Train_Inputs = All_Inputs[select_indexs_train]
Train_command_class = All_command_class[select_indexs_train]
sum(Train_command_class)

array([30., 30., 30., 30., 30., 30., 30., 25., 30., 30.], dtype=float32)

In [40]:
high_acc = 0
for run in range(0,10):
    # feature extraction layers
    resnet_model = ResNet50(input_shape=(224, 224,3), include_top=False, weights="imagenet")
    for layer in resnet_model.layers:
        layer.trainable = False

    d = resnet_model.output.shape[-1] # dimension of last layer


    ###################### model 2 ###################### 
    layer_2_0 = tf.keras.layers.Dense(d,name="weight_2")(resnet_model.output) #times weight before flatten
    layer_2_1 = tf.keras.layers.Flatten(name='flatten_2')(layer_2_0)

    Dense_2_1 = tf.keras.layers.Dense(shape_2_1, activation=actv_fun_2_1,name='fc2_1')
    layer_2_2  = Dense_2_1(layer_2_1)
    Dense_2_2 = tf.keras.layers.Dense(shape_2_2, activation=actv_fun_2_2,name='fc2_2')
    layer_2_3  = Dense_2_2(layer_2_2)

    Dense_2_3 = tf.keras.layers.Dense(10, activation='softmax',name='command_output')
    out_layer_2 = Dense_2_3(layer_2_3)


    resnet_model = tf.keras.Model(resnet_model.input, out_layer_2)


    # resnet_model.summary() 


    ##################### training ############################
    opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
    callbacks = [EarlyStopping(monitor = 'val_accuracy', patience=5,restore_best_weights=True)]
    resnet_model.compile(optimizer=opt, loss="categorical_crossentropy",metrics=['accuracy'])

    
    start = timeit.default_timer()
       
    history1 = resnet_model.fit(Train_Inputs,Train_command_class,
                                validation_data=(Val_Inputs,Val_command_class),
                                callbacks=callbacks,
                                batch_size=64,
                                epochs=10)
    
    for layer in resnet_model.layers[0:175]:
        layer.trainable = True
        
    history2 = resnet_model.fit(Train_Inputs,Train_command_class,
                            validation_data=(Val_Inputs,Val_command_class), 
                           callbacks=callbacks,
                           batch_size=64,
                           epochs=10)

    stop = timeit.default_timer()
    run_time = stop - start
    
    ##################### test performance ############################
    predictions = resnet_model.predict(Test_Inputs)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class
    acc_p15_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)


    # test on p1
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_1)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_1, axis=-1)
    acc_p1_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p2
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_2)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_2, axis=-1)
    acc_p2_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p3 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_3)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_3, axis=-1)
    acc_p3_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p4
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_4)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_4, axis=-1)
    acc_p4_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p5

    ## command classification
    predictions = resnet_model.predict(Test_Inputs_5)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_5, axis=-1)
    acc_p5_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p6 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_6)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class_6
    acc_p6 = metrics.accuracy_score(true_classes, predicted_classes).round(4)                

    # test on p7 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_7)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class_7
    acc_p7 = metrics.accuracy_score(true_classes, predicted_classes).round(4)    

    # test on p8 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_8)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class_8
    acc_p8 = metrics.accuracy_score(true_classes, predicted_classes).round(4)    

    Perfomance = Perfomance.append({'Model': "Subjet_5",'Size':'30','Time':run_time,'Command_Acc_p15':acc_p15_c,
                                    'Command_Acc_p1':acc_p1_c,'Command_Acc_p2':acc_p2_c,
                                    'Command_Acc_p3':acc_p3_c,'Command_Acc_p4':acc_p4_c,'Command_Acc_p5':acc_p5_c,
                                    'Acc_p6':acc_p6,'Acc_p7':acc_p7,'Acc_p8':acc_p8}, ignore_index=True)


    if high_acc < acc_p15_c :
        resnet_model.save('Initial_subject_5_model_size30_0608.h5')
        high_acc = acc_p15_c 
        best_index = run
        pd.DataFrame.from_dict(history1.history).to_csv('subject_5_size30_history1_0608.csv',index=False)
        pd.DataFrame.from_dict(history2.history).to_csv('subject_5_size30_history2_0608.csv',index=False)
        
    del resnet_model
    run = run + 1

Epoch 1/10
5/5 [==============================] - 2s 358ms/step - loss: 2.2989 - accuracy: 0.1831 - val_loss: 2.0036 - val_accuracy: 0.6000
Epoch 2/10
5/5 [==============================] - 1s 166ms/step - loss: 1.9207 - accuracy: 0.7898 - val_loss: 1.8357 - val_accuracy: 0.9600
Epoch 3/10
5/5 [==============================] - 0s 87ms/step - loss: 1.7789 - accuracy: 0.9627 - val_loss: 1.7300 - val_accuracy: 0.9600
Epoch 4/10
5/5 [==============================] - 1s 169ms/step - loss: 1.6809 - accuracy: 0.9695 - val_loss: 1.6421 - val_accuracy: 0.9800
Epoch 5/10
5/5 [==============================] - 1s 164ms/step - loss: 1.6022 - accuracy: 0.9797 - val_loss: 1.5716 - val_accuracy: 0.9900
Epoch 6/10
5/5 [==============================] - 0s 82ms/step - loss: 1.5308 - accuracy: 0.9898 - val_loss: 1.5049 - val_accuracy: 0.9900
Epoch 7/10
5/5 [==============================] - 1s 159ms/step - loss: 1.4666 - accuracy: 0.9898 - val_loss: 1.4373 - val_accuracy: 1.0000
Epoch 8/10
5/5 [======

Epoch 3/10
5/5 [==============================] - 1s 165ms/step - loss: 1.2955 - accuracy: 0.9932 - val_loss: 1.2641 - val_accuracy: 1.0000
Epoch 4/10
5/5 [==============================] - 0s 86ms/step - loss: 1.2219 - accuracy: 0.9966 - val_loss: 1.1978 - val_accuracy: 1.0000
Epoch 5/10
5/5 [==============================] - 0s 84ms/step - loss: 1.1584 - accuracy: 0.9966 - val_loss: 1.1336 - val_accuracy: 1.0000
Epoch 6/10
5/5 [==============================] - 0s 80ms/step - loss: 1.0962 - accuracy: 0.9966 - val_loss: 1.0719 - val_accuracy: 1.0000
Epoch 7/10
5/5 [==============================] - 0s 90ms/step - loss: 1.0362 - accuracy: 1.0000 - val_loss: 1.0151 - val_accuracy: 1.0000
Epoch 8/10
5/5 [==============================] - 1s 154ms/step - loss: 0.9800 - accuracy: 1.0000 - val_loss: 0.9595 - val_accuracy: 1.0000
Epoch 1/10
5/5 [==============================] - 2s 316ms/step - loss: 2.1841 - accuracy: 0.3017 - val_loss: 1.7969 - val_accuracy: 0.8100
Epoch 2/10
5/5 [========

Epoch 8/10
5/5 [==============================] - 1s 156ms/step - loss: 1.0118 - accuracy: 0.9966 - val_loss: 0.9850 - val_accuracy: 1.0000
Epoch 1/10
5/5 [==============================] - 1s 176ms/step - loss: 1.3639 - accuracy: 0.9864 - val_loss: 1.3096 - val_accuracy: 1.0000
Epoch 2/10
5/5 [==============================] - 0s 81ms/step - loss: 1.2397 - accuracy: 0.9932 - val_loss: 1.1902 - val_accuracy: 1.0000
Epoch 3/10
5/5 [==============================] - 0s 83ms/step - loss: 1.1263 - accuracy: 0.9966 - val_loss: 1.0842 - val_accuracy: 1.0000
Epoch 4/10
5/5 [==============================] - 0s 81ms/step - loss: 1.0322 - accuracy: 0.9966 - val_loss: 1.0033 - val_accuracy: 1.0000
Epoch 5/10
5/5 [==============================] - 0s 84ms/step - loss: 0.9521 - accuracy: 0.9966 - val_loss: 0.9278 - val_accuracy: 1.0000
Epoch 6/10
5/5 [==============================] - 1s 160ms/step - loss: 0.8774 - accuracy: 0.9966 - val_loss: 0.8551 - val_accuracy: 1.0000
Epoch 1/10
5/5 [========

In [41]:
resnet_model = tf.keras.models.load_model('Initial_subject_5_model_size30_0608.h5')
predictions = resnet_model.predict(Test_Inputs_5)
predicted_classes =  np.argmax(predictions, axis=1)
acc_c = round(sum(x == y for x, y in zip(np.argmax(Test_command_class_5, axis=1), predicted_classes)) / len(np.argmax(Test_command_class_5, axis=1)),4)
print(acc_c)

0.99


In [42]:
Perfomance

,Model,Time,Command_Acc_p15,Command_Acc_p1,Command_Acc_p2,Command_Acc_p3,Command_Acc_p4,Command_Acc_p5,Acc_p6,Acc_p7,Acc_p8,Size
0,Subjet_1,15.269878,0.6243,0.9369,0.3889,0.5391,0.6239,0.63,0.38,0.384,0.2673,10
1,Subjet_1,12.062418,0.5709,0.9189,0.3148,0.4696,0.5413,0.61,0.33,0.320,0.2673,10
2,Subjet_1,13.520935,0.6335,0.9550,0.3889,0.5478,0.6514,0.62,0.48,0.384,0.2475,10
3,Subjet_1,14.664869,0.6225,0.9550,0.2685,0.5217,0.6789,0.69,0.26,0.456,0.2970,10
4,Subjet_1,13.347304,0.6188,0.9279,0.3796,0.5652,0.5780,0.64,0.39,0.416,0.2574,10
...,...,...,...,...,...,...,...,...,...,...,...,...
145,Subjet_5,14.699057,0.5985,0.5495,0.2685,0.5478,0.6697,0.99,0.30,0.248,0.2673,30
146,Subjet_5,14.681322,0.6354,0.5135,0.2500,0.7304,0.7156,0.99,0.41,0.352,0.2673,30
147,Subjet_5,13.017065,0.5617,0.4505,0.2037,0.5217,0.6881,0.98,0.31,0.344,0.3168,30
148,Subjet_5,12.889397,0.6464,0.5856,0.2407,0.6087,0.8349,0.99,0.35,0.328,0.3267,30


In [43]:
best_index

2

In [44]:
Perfomance.to_csv('Performance_0608_single_model.csv',index=False)